# [STARTER] Exercise - Building an Agentic RAG System

In this exercise, you will build an Agentic RAG (Retrieval-Augmented Generation) system that 
combines the power of AI agents with traditional RAG pipelines. You'll create an agent that 
can decide when and how to retrieve information from different sources, including vector 
databases, web search, and other tools.


## Challenge

Your challenge is to create an Agentic RAG system that can:

- Build a RAG pipeline as a tool that can be used by the agent
- Create an agent that can decide which tool to use based on the query
- Handle different types of queries intelligently
- Combine information from multiple sources when needed


## Setup
First, let's import the necessary libraries:

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
from typing import List
from dotenv import load_dotenv

from lib.agents import Agent
from lib.llm import LLM
from lib.state_machine import Run
from lib.messages import BaseMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager, CorpusLoaderService
from lib.rag import RAG

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Load data to Vector DB

In [5]:
db = VectorStoreManager(OPENAI_API_KEY)
db

VectorStoreManager():<chromadb.api.client.Client object at 0x7e5b5ff03620>

In [6]:
loader_service = CorpusLoaderService(db)

In [7]:
rag_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

In [8]:
# DONE: Add the games pdf file path with the extension .pdf
# And define a store name in load_pdf() method

games_market_rag = RAG(
    llm=rag_llm,
    vector_store = loader_service.load_pdf("gaming_industry", "TheGamingIndustry2024.pdf")
)

VectorStore `gaming_industry` ready!
Pages from `TheGamingIndustry2024.pdf` added!


In [9]:
result:Run = games_market_rag.invoke(
    "What's the  state of virtual reality"
)
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
The state of virtual reality (VR) in 2024 is characterized by significant advancements and immersive experiences that enhance gaming. VR technology allows players to fully engage in digital environments, offering realistic interactions and sensory engagement. Games like "Half-Life: Alyx" exemplify this trend by providing high-fidelity graphics and intuitive controls that immerse players in compelling narratives. Additionally, VR is being integrated into educational settings, enhancing learning through immersive experiences. Overall, VR is reshaping the gaming landscape and expanding its applications beyond entertainment into areas like education and mental health.


In [10]:
# DONE: Add the electric vehicles pdf file path with the extension .pdf
# And define a store name in load_pdf() method

electric_vehicles_rag = RAG(
    llm=rag_llm,
    vector_store = loader_service.load_pdf("global_ev_outlook", "GlobalEVOutlook2025.pdf")
)

VectorStore `global_ev_outlook` ready!
Pages from `GlobalEVOutlook2025.pdf` added!


In [11]:
result:Run = electric_vehicles_rag.invoke("What was the number of electric car sales and their market share in Brazil in 2024?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
In 2024, Brazil saw nearly 125,000 electric car sales, which represented a market share of 6.5%.


## Tools

In a simple form, Agentic RAG can act like a router, choosing between multiple external sources to retrieve relevant information. These sources aren't limited to databases, they can also include tools like web search or APIs for services such as Slack or email.

In this case it will choose between two collections.

In [12]:
# DONE: Define a tool that returns result.get_final_state()["answer"]
# DO NOT Forget about defining the tool docstrings
@tool
def search_global_ev_collection(query):
    """
    Ask questions about global EV outlook.

    args:
        query (str): Search query
    """
    result:Run = electric_vehicles_rag.invoke(query)
    return result.get_final_state()["answer"]

In [13]:
# DONE: Define a tool that returns result.get_final_state()["answer"]
# DO NOT Forget about defining the tool docstrings
@tool
def search_games_market_report_collection(query):
    """
    Ask questions about the game development industry.

    args:
        query (str): Search query
    """
    result:Run = games_market_rag.invoke(query)
    return result.get_final_state()["answer"]

In [14]:
# DONE: Add the tools you have defined and the instructions to your agent
agentic_rag = Agent(
    model_name="gpt-4o-mini",
    tools=[search_global_ev_collection, search_games_market_report_collection],    
    instructions="""
You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions.
Reason about the response, change the query and call the tool again if needed in order to get better results.
Always explain your reasoning for tool selection and provide comprehensive answers.
"""
)

In [15]:
def print_messages(messages: List[BaseMessage]):
    for m in messages: 
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run

In [16]:
run_1 = agentic_rag.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
    session_id="oscar",
)

print("\nMessages from run 1:")
messages = run_1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = 
You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions.
Reason about the response, change the query and call the tool again if needed in order to get better results.
Always explain your reasoning for tool selection and provide comprehensive answers.
, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = The 2025 Oscars have not yet occurred, as they are scheduled for March 2025. Therefore, I cannot provide the winner for the Oscar for International Movie in 2025. If you have any other questions or need information about previous Oscar winners or nominations, feel free to ask!, tool_calls = None)


In [17]:
run_2 = agentic_rag.invoke(
    query= (
        "Which two countries accounted for most of the electric car exports from " 
        "the Asia Pacific region (excluding China) in 2024?"
    ),
    session_id="electric_car",
)

print("\nMessages from run 2:")
messages = run_2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = 
You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions.
Reason about the response, change the query and call the tool again if needed in order to get better results.
Always explain your reasoning for tool selection and provide comprehensive answers.
, tool_calls = None)
 -> (role = user, content = Which two countries accounted for most of the electric car exports from the Asia Pacific region (excluding China) in 2024?, tool_calls = Non

In [18]:
run_3 = agentic_rag.invoke(
    query= (
        "Why is generative AI seen more as an accelerator than a replacement in game development?"
    ),
    session_id="games",
)

print("\nMessages from run 3:")
messages = run_3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = 
You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions.
Reason about the response, change the query and call the tool again if needed in order to get b